# Creating fake data to train the AGD
To be able to run fit your data, you first need to train the Autonomous Gaussian Decomposition (AGD) using fake data that has a similar composition to the one you actually want to fit. This is so the code knows what you are expecting and is able to look for that. 

To see the spectra of the cube I want to fit, and therefore the results I am expecting of the fitting/decomposition, I used CARTA. Then I created the fake data using the code GaussPy has on their website:

In [1]:
# Create training dataset with Gaussian profiles

import numpy as np
import pickle

# Specify the number of spectral channels (NCHANNELS)
NCHANNELS = 199

# Specify the number of spectra (NSPECTRA)
NSPECTRA = 256

# Estimate of the root-mean-square uncertainty per channel (RMS)
RMS = 0.009

# Estimate the number of components
NCOMPS = 2

# Specify the min-max range of possible properties of the Gaussian function parameters:
AMP_lims = [1.5, 8.8]
FWHM_lims = [5, 20] # channels
MEAN_lims = [50, 100] # channels

# Indicate whether the data created here will be used as a training set
# (a.k.a. decide to store the "true" answers or not at the end)
TRAINING_SET = True

# Specify the pickle file to store the results in
FILENAME = 'cube_training_data.pickle'

In [2]:
# Create training dataset with Gaussian profiles -cont-

def gaussian(amp, fwhm, mean):
    return lambda x: amp * np.exp(-4. * np.log(2) * (x-mean)**2 / fwhm**2)

# Initialize
data = {}
chan = np.arange(NCHANNELS)
errors = np.ones(NCHANNELS) * RMS

# Begin populating data
for i in range(NSPECTRA):
    spectrum_i = np.random.randn(NCHANNELS) * RMS

    amps = []
    fwhms = []
    means = []

    for comp in range(NCOMPS):
        # Select random values for components within specified ranges
        a = np.random.uniform(AMP_lims[0], AMP_lims[1])
        w = np.random.uniform(FWHM_lims[0], FWHM_lims[1])
        m = np.random.uniform(MEAN_lims[0], MEAN_lims[1])

        # Add Gaussian profile with the above random parameters to the spectrum
        spectrum_i += gaussian(a, w, m)(chan)

        # Append the parameters to initialized lists for storing
        amps.append(a)
        fwhms.append(w)
        means.append(m)

    # Enter results into AGD dataset
    data['data_list'] = data.get('data_list', []) + [spectrum_i]
    data['x_values'] = data.get('x_values', []) + [chan]
    data['errors'] = data.get('errors', []) + [errors]

    # If training data, keep answers
    if TRAINING_SET:
        data['amplitudes'] = data.get('amplitudes', []) + [amps]
        data['fwhms'] = data.get('fwhms', []) + [fwhms]
        data['means'] = data.get('means', []) + [means]

# Dump synthetic data into specified filename
pickle.dump(data, open(FILENAME, 'wb'))

# Training the AGD with the fake data
After training the AGD, it will give you the best smoothing parameters (alpha 1 and 2) it can find for your data. These values are then used to fit/decompose your actual data in 'gausspy_reading_and_fitting_cube.ipynb'.

In [ ]:
# Select the optimal value of alpha by training the AGD algorithm

import gausspy.gp as gp

# Set necessary parameters
FILENAME_TRAIN = 'cube_training_data.pickle'
snr_thresh = 3
alpha1_initial = -0.44
alpha2_initial = 2.12

g = gp.GaussianDecomposer()                                                                                     

# Next, load the training dataset for analysis:
g.load_training_data(FILENAME_TRAIN)

# Set GaussPy parameters
g.set('phase', 'two')
g.set('SNR_thresh', [snr_thresh, snr_thresh])

# Train AGD starting with initial guess for alpha
g.train(alpha1_initial = alpha1_initial, alpha2_initial = alpha2_initial)

Training...

-0.44 0.9 0.061615047332175044 0.0
iter 0: F1=75.2%, alpha=[-0.44, 2.12], p=[0.00, 0.00]  (Convergence testing begins in 20 iterations)

-0.4954535425989575 0.9 0.11448936760197476 -0.04436283407916602
iter 1: F1=75.8%, alpha=[-0.5, 2.16], p=[-0.04, 0.03]  (Convergence testing begins in 19 iterations)

-0.6428568075199008 0.9 0.14304873294637116 -0.11792261193675464
iter 2: F1=77.6%, alpha=[-0.64, 2.23], p=[-0.12, 0.05]  (Convergence testing begins in 18 iterations)

-0.8895232791083896 0.9 0.21542044203515892 -0.19733317727079103
iter 3: F1=81.4%, alpha=[-0.89, 2.36], p=[-0.20, 0.11]  (Convergence testing begins in 17 iterations)

-1.2807348542108237 0.9 0.009962688381992824 -0.3129692600819473
iter 4: F1=87.1%, alpha=[-1.28, 2.52], p=[-0.31, 0.13]  (Convergence testing begins in 16 iterations)

-1.6026705338365645 0.9 -0.0978123213524362 -0.2575485437005927
iter 5: F1=86.8%, alpha=[-1.6, 2.7], p=[-0.26, 0.14]  (Convergence testing begins in 15 iterations)

-1.77218798831

NameError: name 'quit' is not defined